# Introduction to Docker

In [ ]:
gcloud auth list
gcloud config list project

### Task 1. Hello world

In [ ]:
docker run hello-world
docker images
docker ps -a

### Task 2. Build

In [ ]:
cat > Dockerfile <<EOF
# Use an official Node runtime as the parent image
FROM node:lts
# Set the working directory in the container to /app
WORKDIR /app
# Copy the current directory contents into the container at /app
ADD . /app
# Make the container's port 80 available to the outside world
EXPOSE 80
# Run app.js using node when the container launches
CMD ["node", "app.js"]
EOF

In [ ]:
cat > app.js <<EOF
const http = require('http');
const hostname = '0.0.0.0';
const port = 80;
const server = http.createServer((req, res) => {
    res.statusCode = 200;
    res.setHeader('Content-Type', 'text/plain');
    res.end('Hello World\n');
});
server.listen(port, hostname, () => {
    console.log('Server running at http://%s:%s/', hostname, port);
});
process.on('SIGINT', function() {
    console.log('Caught interrupt signal and will exit');
    process.exit();
});
EOF

In [ ]:
docker build -t node-app:0.1 .
docker images

### Task 3. Run

In [ ]:
docker run -p 4000:80 --name my-app node-app:0.1
curl http://localhost:4000

In [ ]:
docker stop my-app && docker rm my-app

docker run -p 4000:80 --name my-app -d node-app:0.1
docker ps
docker psdocker logs b0433130e779

In [ ]:
cat > app.js <<EOF
const http = require('http');
const hostname = '0.0.0.0';
const port = 80;
const server = http.createServer((req, res) => {
    res.statusCode = 200;
    res.setHeader('Content-Type', 'text/plain');
    res.end('Welcome to Cloud\n');
});
server.listen(port, hostname, () => {
    console.log('Server running at http://%s:%s/', hostname, port);
});
process.on('SIGINT', function() {
    console.log('Caught interrupt signal and will exit');
    process.exit();
});
EOF

In [ ]:
docker build -t node-app:0.2 .
docker run -p 8080:80 --name my-app-2 -d node-app:0.2
docker ps
curl http://localhost:8080

### Task 4. Debug

In [ ]:
docker logs -f b0433130e779

docker exec -it b0433130e779 bash

docker inspect b0433130e779
docker inspect --format='{{range .NetworkSettings.Networks}}{{.IPAddress}}{{end}}' b0433130e779

### Task 5. Publish

In [ ]:
gcloud auth configure-docker us-central1-docker.pkg.dev

export PROJECT_ID=$(gcloud config get-value project)

docker build -t us-central1-docker.pkg.dev/$PROJECT_ID/my-repository/node-app:0.2 .
docker images

docker push us-central1-docker.pkg.dev/$PROJECT_ID/my-repository/node-app:0.2

In [ ]:
docker stop $(docker ps -q)
docker rm $(docker ps -aq)

In [ ]:
docker rmi us-central1-docker.pkg.dev/$PROJECT_ID/my-repository/node-app:0.2
docker rmi node:lts
docker rmi -f $(docker images -aq) # remove remaining images
docker images

In [ ]:
docker pull us-central1-docker.pkg.dev/$PROJECT_ID/my-repository/node-app:0.2
docker run -p 4000:80 -d us-central1-docker.pkg.dev/$PROJECT_ID/my-repository/node-app:0.2
curl http://localhost:4000